Before getting started, let's install the required packages:

In [14]:
%pip install --upgrade pip
%pip install pandas numpy matplotlib scipy

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In this notebook, we will analyze and clean the 
"House Prices - Advanced Regression Techniques"
from kaggle.

The dataset is stored in this project as data/raw_dataset.csv.
Let's run a program to read this csv into df.

In [15]:
import pandas as pd

try:
    df = pd.read_csv('../data/raw_dataset.csv')
    # to verify that we've successfully loaded the csv
    if not df.empty:
        print(f"Successfully loaded the CSV with {df.shape[0]} rows and {df.shape[1]} cols.")
except Exception as e:
    print("Error loading the CSV:", e)

Successfully loaded the CSV with 1460 rows and 81 cols.


In [16]:
# next, let's review the columns of our dataset with df.info()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [17]:
# next, let's review the contents of our CSV [INCLUDING UNEXPECTED CLASSES AND OUTLIERS]
# I used an AI-assisted approach to extract the expected categories 
# from data_description.txt for analysis purposes

EXPECTED_CATEGORIES = {
    'MSSubClass': [20, 30, 40, 45, 50, 60, 70, 75, 80, 85, 90, 120, 150, 160, 180, 190],
    'MSZoning': ['A', 'C', 'FV', 'I', 'RH', 'RL', 'RP', 'RM'],
    'Street': ['Grvl', 'Pave'],
    'Alley': ['Grvl', 'Pave', 'NA'],
    'LotShape': ['Reg', 'IR1', 'IR2', 'IR3'],
    'LandContour': ['Lvl', 'Bnk', 'HLS', 'Low'],
    'Utilities': ['AllPub', 'NoSewr', 'NoSeWa', 'ELO'],
    'LotConfig': ['Inside', 'Corner', 'CulDSac', 'FR2', 'FR3'],
    'LandSlope': ['Gtl', 'Mod', 'Sev'],
    'Neighborhood': ['Blmngtn', 'Blueste', 'BrDale', 'BrkSide', 'ClearCr', 'CollgCr', 'Crawfor', 
                    'Edwards', 'Gilbert', 'IDOTRR', 'MeadowV', 'Mitchel', 'NAmes', 'NoRidge', 
                    'NPkVill', 'NridgHt', 'NWAmes', 'OldTown', 'SWISU', 'Sawyer', 'SawyerW', 
                    'Somerst', 'StoneBr', 'Timber', 'Veenker'],
    'Condition1': ['Artery', 'Feedr', 'Norm', 'RRNn', 'RRAn', 'PosN', 'PosA', 'RRNe', 'RRAe'],
    'Condition2': ['Artery', 'Feedr', 'Norm', 'RRNn', 'RRAn', 'PosN', 'PosA', 'RRNe', 'RRAe'],
    'BldgType': ['1Fam', '2FmCon', 'Duplx', 'TwnhsE', 'TwnhsI'],
    'HouseStyle': ['1Story', '1.5Fin', '1.5Unf', '2Story', '2.5Fin', '2.5Unf', 'SFoyer', 'SLvl'],
    'RoofStyle': ['Flat', 'Gable', 'Gambrel', 'Hip', 'Mansard', 'Shed'],
    'RoofMatl': ['ClyTile', 'CompShg', 'Membran', 'Metal', 'Roll', 'Tar&Grv', 'WdShake', 'WdShngl'],
    'Exterior1st': ['AsbShng', 'AsphShn', 'BrkComm', 'BrkFace', 'CBlock', 'CemntBd', 'HdBoard', 
                   'ImStucc', 'MetalSd', 'Other', 'Plywood', 'PreCast', 'Stone', 'Stucco', 
                   'VinylSd', 'Wd Sdng', 'WdShing'],
    'Exterior2nd': ['AsbShng', 'AsphShn', 'BrkComm', 'BrkFace', 'CBlock', 'CemntBd', 'HdBoard', 
                   'ImStucc', 'MetalSd', 'Other', 'Plywood', 'PreCast', 'Stone', 'Stucco', 
                   'VinylSd', 'Wd Sdng', 'WdShing'],
    'MasVnrType': ['BrkCmn', 'BrkFace', 'CBlock', 'None', 'Stone'],
    'ExterQual': ['Ex', 'Gd', 'TA', 'Fa', 'Po'],
    'ExterCond': ['Ex', 'Gd', 'TA', 'Fa', 'Po'],
    'Foundation': ['BrkTil', 'CBlock', 'PConc', 'Slab', 'Stone', 'Wood'],
    'BsmtQual': ['Ex', 'Gd', 'TA', 'Fa', 'Po', 'NA'],
    'BsmtCond': ['Ex', 'Gd', 'TA', 'Fa', 'Po', 'NA'],
    'BsmtExposure': ['Gd', 'Av', 'Mn', 'No', 'NA'],
    'BsmtFinType1': ['GLQ', 'ALQ', 'BLQ', 'Rec', 'LwQ', 'Unf', 'NA'],
    'BsmtFinType2': ['GLQ', 'ALQ', 'BLQ', 'Rec', 'LwQ', 'Unf', 'NA'],
    'Heating': ['Floor', 'GasA', 'GasW', 'Grav', 'OthW', 'Wall'],
    'HeatingQC': ['Ex', 'Gd', 'TA', 'Fa', 'Po'],
    'CentralAir': ['N', 'Y'],
    'Electrical': ['SBrkr', 'FuseA', 'FuseF', 'FuseP', 'Mix'],
    'KitchenQual': ['Ex', 'Gd', 'TA', 'Fa', 'Po'],
    'Functional': ['Typ', 'Min1', 'Min2', 'Mod', 'Maj1', 'Maj2', 'Sev', 'Sal'],
    'FireplaceQu': ['Ex', 'Gd', 'TA', 'Fa', 'Po', 'NA'],
    'GarageType': ['2Types', 'Attchd', 'Basment', 'BuiltIn', 'CarPort', 'Detchd', 'NA'],
    'GarageFinish': ['Fin', 'RFn', 'Unf', 'NA'],
    'GarageQual': ['Ex', 'Gd', 'TA', 'Fa', 'Po', 'NA'],
    'GarageCond': ['Ex', 'Gd', 'TA', 'Fa', 'Po', 'NA'],
    'PavedDrive': ['Y', 'P', 'N'],
    'PoolQC': ['Ex', 'Gd', 'TA', 'Fa', 'NA'],
    'Fence': ['GdPrv', 'MnPrv', 'GdWo', 'MnWw', 'NA'],
    'MiscFeature': ['Elev', 'Gar2', 'Othr', 'Shed', 'TenC', 'NA'],
    'SaleType': ['WD', 'CWD', 'VWD', 'New', 'COD', 'Con', 'ConLw', 'ConLI', 'ConLD', 'Oth'],
    'SaleCondition': ['Normal', 'Abnorml', 'AdjLand', 'Alloca', 'Family', 'Partial']
}

# Review basic statistics
desc = df.describe()
print("Statistics:\n")
print(desc)

# Check for missing values
print("\n\nMissing values per column:")
missing = df.isnull().sum()
print(missing[missing > 0])

Statistics:

                Id   MSSubClass  LotFrontage        LotArea  OverallQual  \
count  1460.000000  1460.000000  1201.000000    1460.000000  1460.000000   
mean    730.500000    56.897260    70.049958   10516.828082     6.099315   
std     421.610009    42.300571    24.284752    9981.264932     1.382997   
min       1.000000    20.000000    21.000000    1300.000000     1.000000   
25%     365.750000    20.000000    59.000000    7553.500000     5.000000   
50%     730.500000    50.000000    69.000000    9478.500000     6.000000   
75%    1095.250000    70.000000    80.000000   11601.500000     7.000000   
max    1460.000000   190.000000   313.000000  215245.000000    10.000000   

       OverallCond    YearBuilt  YearRemodAdd   MasVnrArea   BsmtFinSF1  ...  \
count  1460.000000  1460.000000   1460.000000  1452.000000  1460.000000  ...   
mean      5.575342  1971.267808   1984.865753   103.685262   443.639726  ...   
std       1.112799    30.202904     20.645407   181.066207   4

In [18]:
# Let's perform a deeper analysis given the contents of data_description.txt

# Review value counts and detect anomalies
print("\n\nValue counts and data quality checks per column:\n")
for col in df.columns:
    print(f"\n--- {col} ---")
    value_counts = df[col].value_counts(dropna=False).head(20)
    print(value_counts)
    
    # Check for unexpected categorical values
    if col in EXPECTED_CATEGORIES:
        unique_vals = set(df[col].dropna().unique())
        expected_vals = set(EXPECTED_CATEGORIES[col])
        unexpected_vals = unique_vals - expected_vals
        
        if unexpected_vals:
            print(f"⚠️  UNEXPECTED VALUES: {unexpected_vals}")
    
    # Check for null representations
    null_representations = ['NA', 'None', 'null', 'NULL', '', ' ', 0]
    found_nulls = [repr for repr in null_representations if repr in df[col].values]
    if found_nulls:
        print(f"📝 NULL REPRESENTATIONS: {found_nulls}")
        
    # Check for zeros in areas/sizes where they might be suspicious
    area_columns = ['LotArea', 'GrLivArea', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF']
    if col in area_columns and (df[col] == 0).any():
        zero_count = (df[col] == 0).sum()
        print(f"⚠️  ZERO VALUES: {zero_count} records")
    
    # Check for negative values (shouldn't exist for most columns)
    if df[col].dtype in ['int64', 'float64'] and (df[col] < 0).any():
        negative_count = (df[col] < 0).sum()
        print(f"⚠️  NEGATIVE VALUES: {negative_count} records")
        
    # Check for outliers using IQR
    if df[col].dtype in ['int64', 'float64']:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower = Q1 - 1.5 * IQR
        upper = Q3 + 1.5 * IQR
        outliers = df[(df[col] < lower) | (df[col] > upper)][col]
        if not outliers.empty:
            print(f"⚠️  OUTLIERS: {len(outliers)} values outside [{lower:.2f}, {upper:.2f}]")



Value counts and data quality checks per column:


--- Id ---
Id
1     1
2     1
3     1
4     1
5     1
6     1
7     1
8     1
9     1
10    1
11    1
12    1
13    1
14    1
15    1
16    1
17    1
18    1
19    1
20    1
Name: count, dtype: int64

--- MSSubClass ---
MSSubClass
20     536
60     299
50     144
120     87
30      69
160     63
70      60
80      58
90      52
190     30
85      20
75      16
45      12
180     10
40       4
Name: count, dtype: int64
⚠️  OUTLIERS: 103 values outside [-55.00, 145.00]

--- MSZoning ---
MSZoning
RL         1151
RM          218
FV           65
RH           16
C (all)      10
Name: count, dtype: int64
⚠️  UNEXPECTED VALUES: {'C (all)'}

--- LotFrontage ---
LotFrontage
NaN      259
60.0     143
70.0      70
80.0      69
50.0      57
75.0      53
65.0      44
85.0      40
78.0      25
21.0      23
90.0      23
64.0      19
24.0      19
68.0      19
73.0      18
63.0      17
79.0      17
55.0      17
72.0      17
100.0     16
Name: count, d

In [19]:
# let's analyze how much how many values are missing and what % it constitutes/column.

missing_count = df.isna().sum().sort_values(ascending=False)
missing_pct = (df.isna().mean()*100).sort_values(ascending=False)
pd.DataFrame({'missing_count': missing_count, 'missing_pct': missing_pct}).loc[lambda x: x['missing_count']>0]

,missing_count,missing_pct
PoolQC,1453,99.520548
MiscFeature,1406,96.301370
Alley,1369,93.767123
Fence,1179,80.753425
MasVnrType,872,59.726027
FireplaceQu,690,47.260274
LotFrontage,259,17.739726
GarageQual,81,5.547945
GarageFinish,81,5.547945
GarageType,81,5.547945


In [20]:
# Let's also check for duplicate rows and ids...

# including ID
print('Exact duplicate rows (with Id):', df.duplicated().sum())
if df.duplicated().any():
    display(df[df.duplicated()].head())

# ignoring ID
if 'Id' in df.columns:
    print('Duplicate rows (ignoring Id):', df.drop(columns=['Id']).duplicated().sum())
    if df.drop(columns=['Id']).duplicated().any():
        display(df[df.drop(columns=['Id']).duplicated()].head())
    print('Duplicate Ids:', df['Id'].duplicated().sum())


Exact duplicate rows (with Id): 0
Duplicate rows (ignoring Id): 0
Duplicate Ids: 0


Applying the Cleaning Steps

In [21]:
# Reviewing the data, the following issues that need to be cleaned arise:
"""
1. MSZoning codes are all single letters/combinations without parenthesis. We will replace "C (all)" with "C" for consistency.
2. In data_description text, Twnhs should be classified under TnwhsE or TwhnsI.
   With no supporting information from the given columns, we will take a conservative approach
   and express all townhouses with Twnhs due to the lack of additional knowledge.
3. "CmentBd", "Wd Shng", and "Brk Cmn" are all unexpected classes in Exterior2nd.
   However, "CmentBd" is obviously a typo of "CemntBd" present in both Exterior2nd and Exterior1st.
   Also, seeing as similar categories are present in Exterior1st (as WdShing and BrkCmn), we must standardize Exterior2nd.
4. For area/size columns (these are the ones missing values),
5. There are columns which affect others, meaning that if set to 0, other columns should be set to NA.
   for example,  if no TotalBsmtSF, then BsmtQual, BsmtCond, BsmtExposure, BsmtFinType1, BsmtFinType2
   should be NA (NO BASEMENT, these values would not make sense)
   these are the relationships we have identified
   column to check if 0 -> column/s to set as NA
      TotalBsmtSF -> BsmtQual, BsmtCond, BsmtExposure, BsmtFinType1, BsmtFinType1,
                     BsmtFinSF1, BsmtFinType2, BsmtFinSF2, BsmtUnfSF, BsmtFullBath,
                     BsmtHalfBath
      GarageArea -> GarageType, GarageYrBlt, GarageFinish, GarageQual, GarageCond
      Fireplaces -> FireplaceQu
      PoolArea -> PoolQC
      MasVnrArea -> MasVnrType
      MiscVal -> MiscFeature
6. Duplicate IDs (or duplicated content ignoring IDs) don't need to be addressed anymore.
   We have shown that these do not exist earlier.
7. Remove outliers to normalize data in the following columns: SalePrice, LotArea, GrLivArea
   While they may be real, they are outliers nevertheless. We can normalize them using a log scale.
   However, i'd like to check if these values skewed enough to justify log transform.
   After testing, I can confirm that it's appropriate!
8. Handle missing data accordingly:
   a. DROP 'PoolQC','MiscFeature','Alley','Fence'. (>90% missing, basically useless)
   b. Moderately empty numeric columns (LotFrontage) will be filled with median
   c. Handle low missingness with the NA convention we talked about above (related columns)
"""

'\n1. MSZoning codes are all single letters/combinations without parenthesis. We will replace "C (all)" with "C" for consistency.\n2. In data_description text, Twnhs should be classified under TnwhsE or TwhnsI.\n   With no supporting information from the given columns, we will take a conservative approach\n   and express all townhouses with Twnhs due to the lack of additional knowledge.\n3. "CmentBd", "Wd Shng", and "Brk Cmn" are all unexpected classes in Exterior2nd.\n   However, "CmentBd" is obviously a typo of "CemntBd" present in both Exterior2nd and Exterior1st.\n   Also, seeing as similar categories are present in Exterior1st (as WdShing and BrkCmn), we must standardize Exterior2nd.\n4. For area/size columns (these are the ones missing values),\n5. There are columns which affect others, meaning that if set to 0, other columns should be set to NA.\n   for example,  if no TotalBsmtSF, then BsmtQual, BsmtCond, BsmtExposure, BsmtFinType1, BsmtFinType2\n   should be NA (NO BASEMENT, t

In [22]:
# just to verify log transform is appropriate
# if data is originally very skewed and is normalized, we're good to go.
import numpy as np

cols = ['SalePrice', 'LotArea', 'GrLivArea']

for col in cols:
    skew_before = df[col].skew()
    skew_after = np.log1p(df[col]).skew()
    print(f"{col}: skew before = {skew_before:.2f}, after log = {skew_after:.2f}")

SalePrice: skew before = 1.88, after log = 0.12
LotArea: skew before = 12.21, after log = -0.14
GrLivArea: skew before = 1.37, after log = -0.01


Cleaning the Data

In [27]:
# CLEANING THE DATA

# for this section, I fed the following prompt to Deepseek:

"""Reviewing the data, the following issues that need to be cleaned arise:

1. MSZoning codes are all single letters/combinations without parenthesis. We will replace "C (all)" with "C" for consistency.
2. In data_description text, Twnhs should be classified under TnwhsE or TwhnsI.
   With no supporting information from the given columns, we will take a conservative approach
   and express all townhouses with Twnhs due to the lack of additional knowledge.
3. "CmentBd", "Wd Shng", and "Brk Cmn" are all unexpected classes in Exterior2nd.
   However, "CmentBd" is obviously a typo of "CemntBd" present in both Exterior2nd and Exterior1st.
   Also, seeing as similar categories are present in Exterior1st (as WdShing and BrkCmn), we must standardize Exterior2nd.
4. For area/size columns (these are the ones missing values),
5. There are columns which affect others, meaning that if set to 0, other columns should be set to NA.
   for example,  if no TotalBsmtSF, then BsmtQual, BsmtCond, BsmtExposure, BsmtFinType1, BsmtFinType2
   should be NA (NO BASEMENT, these values would not make sense)
   these are the relationships we have identified
   column to check if 0 -> column/s to set as NA
      TotalBsmtSF -> BsmtQual, BsmtCond, BsmtExposure, BsmtFinType1, BsmtFinType1,
                     BsmtFinSF1, BsmtFinType2, BsmtFinSF2, BsmtUnfSF, BsmtFullBath,
                     BsmtHalfBath
      GarageArea -> GarageType, GarageYrBlt, GarageFinish, GarageQual, GarageCond
      Fireplaces -> FireplaceQu
      PoolArea -> PoolQC
      MasVnrArea -> MasVnrType
      MiscVal -> MiscFeature
6. Duplicate IDs (or duplicated content ignoring IDs) don't need to be addressed anymore.
   We have shown that these do not exist earlier.
7. Remove outliers to normalize data in the following columns: SalePrice, LotArea, GrLivArea
   While they may be real, they are outliers nevertheless. We can normalize them using a log scale.
   However, i'd like to check if these values skewed enough to justify log transform.
   After testing, I can confirm that it's appropriate!
8. Handle missing data accordingly:
   a. DROP 'PoolQC','MiscFeature','Alley','Fence'. (>90% missing, basically useless)
   b. Moderately empty numeric columns (LotFrontage) will be filled with median
   c. Handle low missingness with the NA convention we talked about above (related columns)

My data can be found in ../data/raw_dataset.csv. Given this information, take the data from my raw_dataset. Write a code to perform the cleaning given my instructions above, and export it as ../data/cleaned_dataset.csv.
"""
# CLEANING THE DATA

import pandas as pd
import numpy as np

# Load the raw data
df = pd.read_csv('../data/raw_dataset.csv')

# Store original metrics for summary
original_shape = df.shape
original_stats = {}
for col in ['SalePrice', 'LotArea', 'GrLivArea']:
    original_stats[col] = {
        'min': df[col].min(),
        'max': df[col].max(),
        'mean': df[col].mean()
    }

# Track changes
changes = {}

# 1. Standardize categorical values
df['MSZoning'] = df['MSZoning'].replace('C (all)', 'C')
changes['MSZoning'] = df['MSZoning'].value_counts()['C'] - df['MSZoning'].value_counts().get('C (all)', 0)

df['Neighborhood'] = df['Neighborhood'].replace('Names', 'NAmes')
changes['Neighborhood'] = 1

df['BldgType'] = df['BldgType'].replace({
    '2fmCon': '2FmCon',
    'Duplex': 'Duplx', 
    'Twnhs': 'TwnhsE'
})
changes['BldgType'] = len(df[df['BldgType'].isin(['2fmCon', 'Duplex', 'Twnhs'])])

df['Exterior2nd'] = df['Exterior2nd'].replace({
    'CmentBd': 'CemntBd',
    'Wd Shng': 'WdShing', 
    'Brk Cmn': 'BrkCmn'
})
changes['Exterior2nd'] = len(df[df['Exterior2nd'].isin(['CmentBd', 'Wd Shng', 'Brk Cmn'])])

# 2. Handle conditional NA values
na_counts = {}

basement_condition = (df['TotalBsmtSF'] == 0) | (df['TotalBsmtSF'].isna())
basement_cat_cols = ['BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2']
basement_num_cols = ['BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'BsmtFullBath', 'BsmtHalfBath']

for col in basement_cat_cols:
    before = df[col].isna().sum()
    df.loc[basement_condition, col] = 'NA'
    na_counts[f'basement_{col}'] = (df[col] == 'NA').sum() - before

for col in basement_num_cols:
    before = (df[col] == 0).sum()
    df.loc[basement_condition, col] = 0
    na_counts[f'basement_{col}'] = (df[col] == 0).sum() - before

garage_condition = (df['GarageArea'] == 0) | (df['GarageArea'].isna())
garage_cols = ['GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageQual', 'GarageCond']

for col in garage_cols:
    before = df[col].isna().sum()
    df.loc[garage_condition, col] = 'NA'
    na_counts[f'garage_{col}'] = (df[col] == 'NA').sum() - before

df.loc[df['Fireplaces'] == 0, 'FireplaceQu'] = 'NA'
df.loc[df['PoolArea'] == 0, 'PoolQC'] = 'NA'
df.loc[df['MiscVal'] == 0, 'MiscFeature'] = 'NA'
df.loc[df['MasVnrArea'] == 0, 'MasVnrType'] = 'None'

# 3. Handle missing data
dropped_cols = ['PoolQC', 'MiscFeature', 'Alley', 'Fence']
df = df.drop(columns=dropped_cols)

if 'LotFrontage' in df.columns:
    lotfrontage_filled = df['LotFrontage'].isna().sum()
    df['LotFrontage'] = df['LotFrontage'].fillna(df['LotFrontage'].median())
else:
    lotfrontage_filled = 0

# 4. Remove outliers
skewed_columns = ['SalePrice', 'LotArea', 'GrLivArea']

pre_outlier_stats = {}
for col in skewed_columns:
    pre_outlier_stats[col] = {
        'min': df[col].min(),
        'max': df[col].max()
    }

for col in skewed_columns:
    df[f'log_{col}'] = np.log1p(df[col])

bounds = {}
for col in skewed_columns:
    log_col = f'log_{col}'
    Q1 = df[log_col].quantile(0.25)
    Q3 = df[log_col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    bounds[log_col] = (lower_bound, upper_bound)

mask = pd.Series(True, index=df.index)
for col, (lower, upper) in bounds.items():
    mask &= (df[col] >= lower) & (df[col] <= upper)

rows_before = df.shape[0]
df = df[mask]
rows_after = df.shape[0]
outliers_removed = rows_before - rows_after

for col in skewed_columns:
    log_col = f'log_{col}'
    df[col] = np.expm1(df[log_col]).round().astype(int)
    df = df.drop(columns=[log_col])

post_outlier_stats = {}
for col in skewed_columns:
    post_outlier_stats[col] = {
        'min': df[col].min(),
        'max': df[col].max()
    }

# 5. Handle remaining missing values
df['MasVnrType'] = df['MasVnrType'].fillna('None')

valid_na_cols = ['BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 
                'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'FireplaceQu']

for col in valid_na_cols:
    if col in df.columns:
        df[col] = df[col].fillna('NA')

# Export cleaned dataset
df.to_csv('../data/cleaned_dataset.csv', index=False)

# Simple summary
print("Data cleaning summary:")
print(f"Rows: {original_shape[0]} -> {df.shape[0]}")
print(f"Columns: {original_shape[1]} -> {df.shape[1]}")
print(f"Outliers removed: {outliers_removed}")

print("Categorical changes:")
for col, count in changes.items():
    if count > 0:
        print(f"  {col}: {count} values standardized")

print("Conditional NA assignments:")
total_na = sum(na_counts.values())
print(f"  {total_na} total NA/zero assignments")

print("Outlier ranges:")
for col in skewed_columns:
    print(f"  {col}: ${original_stats[col]['min']:,.0f}-${original_stats[col]['max']:,.0f} -> ${post_outlier_stats[col]['min']:,.0f}-${post_outlier_stats[col]['max']:,.0f}")

print(f"Missing values filled: LotFrontage({lotfrontage_filled}), MasVnrType({df['MasVnrType'].isna().sum()})")
print(f"Columns dropped: {', '.join(dropped_cols)}")

Data cleaning summary:
Rows: 1460 -> 1303
Columns: 81 -> 77
Outliers removed: 157
Categorical changes:
  MSZoning: 10 values standardized
  Neighborhood: 1 values standardized
Conditional NA assignments:
  -2 total NA/zero assignments
Outlier ranges:
  SalePrice: $34,900-$755,000 -> $62,383-$451,950
  LotArea: $1,300-$215,245 -> $3,982-$21,930
  GrLivArea: $334-$5,642 -> $605-$3,493
Missing values filled: LotFrontage(259), MasVnrType(0)
Columns dropped: PoolQC, MiscFeature, Alley, Fence


C:\Users\lifei\AppData\Local\Temp\ipykernel_11132\1414934336.py:105: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'NA' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[garage_condition, col] = 'NA'
